In [1]:
alpha = 0.65
f(k) = k.^alpha
u(x) = log(x)
beta = 0.95

#grid for state variable k and action variable s:
grid_max = 2
grid_size = 500
grid = linspace(1e-6, grid_max, grid_size)

linspace(1.0e-6,2.0,500)

In [2]:
C = reshape(f(grid),grid_size,1).-reshape(grid,1,grid_size)
coord = repmat(collect(1:grid_size),1,grid_size) #coordinate matrix 
s_indices = coord[C.>0]
a_indices = transpose(coord)[C.>0]
L = length(a_indices)


118841

In [3]:
R = u(C[C.>0]);

In [4]:
Q = spzeros(L,grid_size)

for i in 1:L
  Q[i,a_indices[i]] = 1
end

In [5]:
using QuantEcon
@time ddp = DiscreteDP(R, Q, beta, s_indices, a_indices);

  1.253554 seconds (564.85 k allocations: 491.262 MB, 2.79% gc time)


In [9]:
@time results = solve(ddp, PFI);

 11.401085 seconds (616 allocations: 4.966 GB, 8.04% gc time)


In [ ]:
v, sigma, num_iter = results.v, results.sigma, results.num_iter

num_iter

In [ ]:
c = f(grid) - grid[sigma]

ab = alpha * beta
c1 = (log(1 - alpha * beta) + log(alpha * beta) * alpha * beta / (1 - alpha * beta)) / (1 - beta)
c2 = alpha / (1 - alpha * beta)

v_star(k) = c1 + c2 * log(k)
c_star(k) = (1 - alpha * beta) * k.^alpha

using PyPlot
fig, (ax1, ax2) = subplots(1, 2, figsize=(14, 4))


ax1[:set_ylim](-40, -32)
ax1[:set_xlim](grid[1], grid[end])
ax2[:set_xlim](grid[1], grid[end])

lb0 = "discrete value function"
ax1[:plot](grid, v, lw=2, alpha=0.6, label=lb0)
lb0 = "continuous value function"
ax1[:plot](grid, v_star(grid), "k-", lw=1.5, alpha=0.8, label=lb0)
ax1[:legend](loc="upper left")

lb1 = "discrete optimal consumption"
ax2[:plot](grid, c, "b-", lw=2, alpha=0.6, label=lb1)
lb1 = "continuous optimal consumption"
ax2[:plot](grid, c_star(grid), "k-", lw=1.5, alpha=0.8, label=lb1)
ax2[:legend](loc="upper left")

maximum(abs(v - v_star(grid)))

maximum(abs(v - v_star(grid))[2:end])

maximum(abs(c - c_star(grid)))